## Определение важнейших метрик
Владельцам сайта для продажи билетов в кино нужна помощь. В датасете предложен некоторый набор метрик, но он не полный. Определите метрики, которые больше всего влияют на доход. Для этого постройте пирамиду метрик и проведите корреляционный анализ.

In [21]:
# импортируем библиотеки
import pandas as pd

In [8]:
# загрузим датасет
df_metrics = pd.read_excel('Домашнее задание.xls', parse_dates = ['Дата'])
df_metrics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 17 columns):
 #   Column                                          Non-Null Count  Dtype         
---  ------                                          --------------  -----         
 0   Дата                                            125 non-null    datetime64[ns]
 1   Пользователи                                    125 non-null    int64         
 2   Новые пользователи                              125 non-null    int64         
 3   Доля новых пользователей                        125 non-null    float64       
 4   Количество платящих пользователей               125 non-null    int64         
 5   Количество платящих пользователей.1             125 non-null    float64       
 6   Количество покупок                              125 non-null    int64         
 7   Количество покупок.1                            125 non-null    float64       
 8   Количество покупок на платящего пользователя    12

Определим сначала пирамиду метрик, чтобы затем искать зависимости. 
1. Бизнес
2. Экономоческий
3. Продуктовый

На первый уровень мы поставим "Выручка, RUB.1"


От чего зависит наша выручка?
- ARPU - Средняя выручка на пользователя, RUB
- ARPPU - Средняя выручка на платящего пользователя, RUB
- DAU - Пользователи
- LTV, LT - в датасете нет этой метрики, и расчитать на текущих данных мы ее не можем, так как у нас нет возможности посчитать LT на этих данных

Мы определили метрики воторого уровня. 
А метрики третьего уровеня мы найдем через корреляции по коэффициенту Пирсона.

Итого сейчас наша пирамида выглядит так:
1. **Бизнес** - Выручка (Revenue)
2. **Экономоческий** - "Средняя выручка на пользователя, RUB" (ARPU), "Средняя выручка на платящего пользователя, RUB" (ARPPU),  "Пользователи" (DAU)
3. **Продуктовы**


In [9]:
# отредактируем таблицу, уберем "повторяющиеся" метрики
df_metrics.drop(columns=[
    'Доля новых пользователей',
    'Количество платящих пользователей.1',
    'Количество покупок.1',
    'Количество покупок на платящего пользователя',
    'Выручка, RUB'
    ], axis=1, inplace=True)

df_metrics.head()

,Дата,Пользователи,Новые пользователи,Количество платящих пользователей,Количество покупок,"Средняя выручка на пользователя, RUB","Средняя выручка на платящего пользователя, RUB",Вернувшиеся,"Выручка, RUB.1","CR воронки покупок, %",Ошибки,CSI
0,2017-12-31,23095,1603,1828,2309,137.544230,1737.737418,21492,3.176584e+06,19,18854,82
1,2017-12-30,29918,2116,2727,3413,133.536032,1465.027869,27802,3.995131e+06,19,17506,88
2,2017-12-29,38953,3029,4219,5142,178.231792,1645.570751,35924,6.942663e+06,18,10910,88
3,2017-12-28,41326,3228,4710,5732,186.466825,1636.078131,38098,7.705928e+06,17,17235,85
4,2017-12-27,34148,2199,3018,3713,134.607649,1523.055666,31949,4.596582e+06,13,14190,88


In [20]:
# построим корреляционную таблицу для выручки
merics_corr = df_metrics.corr()['Выручка, RUB.1']

# отфильтруем данные, выберем только высокий уровень связи
merics_corr[(abs(merics_corr)>= 0.7)
            & (~merics_corr.index.isin(['Выручка, RUB.1']))]


Пользователи                                      0.838978
Новые пользователи                                0.843391
Количество платящих пользователей                 0.935762
Количество покупок                                0.942241
Средняя выручка на пользователя, RUB              0.966441
Средняя выручка на платящего пользователя, RUB    0.801188
Вернувшиеся                                       0.806211
Name: Выручка, RUB.1, dtype: float64

Таким образом мы нашли прямые высокие корреляции между ключевой метрикой для бизнеса - Выручка и другими метриками. 

Наша итоговая пирамида будет выглядеть так:

1. **Бизнес** 
    - "Выручка" (Revenue)
2. **Экономоческий** 
    - "Средняя выручка на пользователя, RUB" (ARPU)
    - "Средняя выручка на платящего пользователя, RUB" (ARPPU)
    - "Пользователи" (DAU)
3. **Продуктовы**
    - "Новые пользователи" 
    - "Количество платящих пользователей"
    - "Количество покупок"
    - "Вернувшиеся"